### Simple database queries

In [39]:
import sqlite3
import pandas as pd

In [40]:
# Load Customers
df = pd.read_csv('../../datasets/archive/olist_customers_dataset.csv')
df.columns = df.columns.str.strip()
con = sqlite3.connect("data/olist.db")
df.to_sql("customers", con)
con.close()

In [41]:
# Load Orders
df = pd.read_csv('../../datasets/archive/olist_orders_dataset.csv')
df.columns = df.columns.str.strip()
con = sqlite3.connect("data/olist.db")
df.to_sql("orders", con)
con.close()

In [42]:
# Load Products
df = pd.read_csv('../../datasets/archive/olist_order_reviews_dataset.csv')
df.columns = df.columns.str.strip()
con = sqlite3.connect("data/olist.db")
df.to_sql("order_reviews", con)
con.close()

In [43]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [44]:
db = SQLDatabase.from_uri("sqlite:///data/olist.db")
llm = OpenAI(temperature=0, verbose=True)

In [45]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [46]:
db_chain.run("""What column names we have in orders?""")



> Entering new SQLDatabaseChain chain...
What column names we have in orders?
SQLQuery: SELECT * FROM orders LIMIT 5;
SQLResult: [(0, 'e481f51cbdc54678b7cc49136f2d6af7', '9ef432eb6251297304e76186b10a928d', 'delivered', '2017-10-02 10:56:33', '2017-10-02 11:07:15', '2017-10-04 19:55:00', '2017-10-10 21:25:13', '2017-10-18 00:00:00'), (1, '53cdb2fc8bc7dce0b6741e2150273451', 'b0830fb4747a6c6d20dea0b8c802d7ef', 'delivered', '2018-07-24 20:41:37', '2018-07-26 03:24:27', '2018-07-26 14:31:00', '2018-08-07 15:27:45', '2018-08-13 00:00:00'), (2, '47770eb9100c2d0c44946d9cf07ec65d', '41ce2a54c0b03bf3443c3d931a367089', 'delivered', '2018-08-08 08:38:49', '2018-08-08 08:55:23', '2018-08-08 13:50:00', '2018-08-17 18:06:29', '2018-09-04 00:00:00'), (3, '949d5b44dbf5de918fe9c16f97b45f8a', 'f88197465ea7920adcdbec7375364d82', 'delivered', '2017-11-18 19:28:06', '2017-11-18 19:45:59', '2017-11-22 13:39:59', '2017-12-02 00:28:42', '2017-12-15 00:00:00'), (4, 'ad21c59c0840e6cb83a9ceb5573f8159', '8ab9790

' The orders table has the following column names: index, order_id, customer_id, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date.'

In [47]:
db_chain.run("""What is the average delivery time for the orders, for each state? Do not limit results.""")



> Entering new SQLDatabaseChain chain...
What is the average delivery time for the orders, for each state? Do not limit results.
SQLQuery: SELECT customer_state, AVG(julianday(order_delivered_customer_date) - julianday(order_purchase_timestamp)) AS avg_delivery_time
FROM orders
INNER JOIN customers
ON orders.customer_id = customers.customer_id
GROUP BY customer_state
SQLResult: [('AC', 21.035712962946853), ('AL', 24.543855361979436), ('AM', 26.425990900413357), ('AP', 27.185067717006792), ('BA', 19.335465824488363), ('CE', 21.266578634927797), ('DF', 12.967568108972717), ('ES', 15.789307110366133), ('GO', 15.606338535454533), ('MA', 21.572976474119596), ('MG', 12.010258342917206), ('MS', 15.618319018474233), ('MT', 18.05575601172604), ('PA', 23.77291691136826), ('PB', 20.426767609615375), ('PE', 18.448322994401824), ('PI', 19.45709826291378), ('PR', 11.991582227413579), ('RJ', 15.310052815554164), ('RN', 19.278156206030438), ('RO', 19.372036513109755), ('RR', 29.387546013983922), ('R

' The average delivery time for orders, for each state, ranges from 8.76 days in Sao Paulo to 29.39 days in Roraima.'

In [54]:
db_chain.run("""
Retrieve 40 samples order reviews that have comment messages and a review score less than 3. 
What are the main complains?
""")



> Entering new SQLDatabaseChain chain...

Retrieve 40 samples order reviews that have comment messages and a review score less than 3. 
What are the main complains?

SQLQuery: 
SELECT review_comment_message 
FROM order_reviews 
WHERE review_score < 3 
AND review_comment_message IS NOT NULL 
LIMIT 40;

SQLResult: [('GOSTARIA DE SABER O QUE HOUVE, SEMPRE RECEBI E ESSA COMPRA AGORA ME DECPCIONOU',), ('Péssimo',), ('Não gostei ! Comprei gato por lebre',), ('Sempre compro pela Internet e a entrega ocorre antes do prazo combinado, que acredito ser o prazo máximo. No stark o prazo máximo já se esgotou e ainda não recebi o produto.',), ('Nada de chegar o meu pedido.',), ('recebi somente 1 controle Midea Split ESTILO.\r\nFaltou Controle Remoto para Ar Condicionado Consul',), ('O produto não chegou no prazo estipulado e causou transtorno, pq programei a viagem de férias do meu filho, baseado no prazo. Moro na Bahia e ele em Cuiabá sozinho. Agora, a casa está vazia. ',), ('Produto muito inferio

' The main complains are about late delivery, wrong product, missing product, low quality product, and not being able to cancel orders.'